# Operaciones con DataFrames

Este notebook demuestra operaciones avanzadas con pandas DataFrames: transformaciones, agrupaciones y combinación de DataFrames.

**Referencia:** [Operaciones con DataFrames](../pandas/python-para-datos/02-operaciones-dataframes.md)

**Objetivos:**
- Aplicar transformaciones con apply(), map(), replace()
- Realizar agrupaciones con groupby()
- Combinar DataFrames con concat() y merge()
- Usar transform() y filter() para operaciones avanzadas

## 1. Importar librerías y cargar datos

In [ ]:
import pandas as pd
import numpy as np

# Cargar datos desde el CSV de ejemplo
df = pd.read_csv('../data/ventas.csv')

print(f"✅ Datos cargados: {len(df)} registros")
print(f"\nColumnas: {df.columns.tolist()}")
df.head()

## 2. Transformaciones con apply()

In [ ]:
# Aplicar función a columna: precio con IVA
df['precio_con_iva'] = df['precio'].apply(lambda x: x * 1.21)
df[['producto', 'precio', 'precio_con_iva']].head()

# Aplicar función personalizada: categorizar precio
def categorizar_precio(precio):
    if precio < 50:
        return 'Económico'
    elif precio < 200:
        return 'Estándar'
    else:
        return 'Premium'

df['tipo_producto'] = df['precio'].apply(categorizar_precio)
print("\nDistribución por tipo de producto:")
print(df['tipo_producto'].value_counts())

## 3. Transformaciones con map() y replace()

In [ ]:
# Mapear valores: crear regiones desde ciudades
mapeo_region = {
    'Madrid': 'Centro',
    'Barcelona': 'Este',
    'Valencia': 'Este',
    'Sevilla': 'Sur'
}
df['region'] = df['ciudad'].map(mapeo_region)
print("Ciudades y regiones:")
print(df[['ciudad', 'region']].drop_duplicates())

# Reemplazar valores
df['categoria_original'] = df['categoria'].copy()  # Guardar original
df['categoria'] = df['categoria'].replace('Ropa', 'Moda')
print("\nCategorías después de replace:")
print(df['categoria'].value_counts())

## 4. Agrupaciones con groupby()

In [ ]:
# Agrupar y agregar: precio promedio por categoría
print("=== PRECIO PROMEDIO POR CATEGORÍA ===")
precio_promedio = df.groupby('categoria')['precio'].mean()
print(precio_promedio)

# Múltiples agregaciones
print("\n=== ESTADÍSTICAS POR CATEGORÍA ===")
estadisticas = df.groupby('categoria').agg({
    'precio': ['mean', 'min', 'max'],
    'total': 'sum',
    'id': 'count'
})
print(estadisticas)

# Agrupar por múltiples columnas
print("\n=== VENTAS POR CATEGORÍA Y CIUDAD ===")
ventas_cat_ciudad = df.groupby(['categoria', 'ciudad'])['total'].sum()
print(ventas_cat_ciudad)

## 5. Transform y filter

In [ ]:
# Transform: mantiene forma original, agrega columna con promedio por grupo
df['precio_promedio_categoria'] = df.groupby('categoria')['precio'].transform('mean')
df['diferencia_precio'] = df['precio'] - df['precio_promedio_categoria']

print("=== COMPARACIÓN PRECIO VS PROMEDIO DE CATEGORÍA ===")
print(df[['producto', 'categoria', 'precio', 'precio_promedio_categoria', 'diferencia_precio']].head(10))

# Filter: filtra grupos (solo categorías con más de 5 productos)
print("\n=== FILTRAR CATEGORÍAS CON MÁS DE 5 PRODUCTOS ===")
df_filtrado = df.groupby('categoria').filter(lambda x: len(x) > 5)
print(f"Registros originales: {len(df)}")
print(f"Registros después de filtrar: {len(df_filtrado)}")
print(f"Categorías que quedaron: {df_filtrado['categoria'].unique()}")

## 6. Combinar DataFrames con concat()

In [ ]:
# Dividir por categoría
df_electronica = df[df['categoria'] == 'Electrónica'].copy()
df_ropa = df[df['categoria'] == 'Ropa'].copy()
df_hogar = df[df['categoria'] == 'Hogar'].copy()

print(f"Electrónica: {len(df_electronica)} registros")
print(f"Ropa: {len(df_ropa)} registros")
print(f"Hogar: {len(df_hogar)} registros")

# Concatenar verticalmente
df_combinado = pd.concat([df_electronica, df_ropa], ignore_index=True)
print(f"\nTotal registros combinados: {len(df_combinado)}")
print(f"Categorías en combinado: {df_combinado['categoria'].unique()}")

## 7. Combinar DataFrames con merge()

In [ ]:
# Crear dos DataFrames relacionados para demostrar merge
# DataFrame 1: Información de productos (sin duplicados)
df_productos = df[['producto', 'categoria', 'precio']].drop_duplicates(subset=['producto']).reset_index(drop=True)
df_productos['producto_id'] = range(1, len(df_productos) + 1)
print("=== DATAFRAME PRODUCTOS ===")
print(df_productos.head())

# DataFrame 2: Información de ventas (simulando IDs diferentes)
df_ventas = df[['id', 'fecha', 'cantidad', 'total', 'ciudad']].copy()
df_ventas['producto_id'] = (df_ventas['id'] % len(df_productos)) + 1
print("\n=== DATAFRAME VENTAS ===")
print(df_ventas.head())

# Inner join: Solo productos que tienen ventas
df_merged = pd.merge(df_productos, df_ventas, on='producto_id', how='inner')
print(f"\n=== RESULTADO MERGE (INNER JOIN) ===")
print(f"Registros después de merge: {len(df_merged)}")
print(df_merged.head())

# Left join: Todos los productos, incluso sin ventas
df_merged_left = pd.merge(df_productos, df_ventas, on='producto_id', how='left')
print(f"\n=== RESULTADO MERGE (LEFT JOIN) ===")
print(f"Registros con left join: {len(df_merged_left)}")
print(f"Productos sin ventas: {df_merged_left['total'].isnull().sum()}")

## 8. Operaciones combinadas: Resumen completo

Este ejemplo combina todas las técnicas aprendidas.

In [ ]:
# Recargar datos limpios
df = pd.read_csv('../data/ventas.csv')

# 1. Agregar columnas calculadas
df['precio_con_iva'] = df['precio'] * 1.21
df['tipo_producto'] = df['precio'].apply(
    lambda x: 'Premium' if x > 200 else ('Estándar' if x >= 50 else 'Económico')
)

# 2. Agregar promedio por categoría usando transform
df['precio_promedio_categoria'] = df.groupby('categoria')['precio'].transform('mean')
df['por_encima_promedio'] = df['precio'] > df['precio_promedio_categoria']

# 3. Crear resumen por categoría
resumen = df.groupby('categoria').agg({
    'precio': ['mean', 'min', 'max'],
    'total': 'sum',
    'producto': 'nunique',  # Productos únicos
    'ciudad': 'nunique'     # Ciudades únicas
}).round(2)

resumen.columns = ['Precio_Promedio', 'Precio_Min', 'Precio_Max', 'Total_Ventas', 'Productos_Unicos', 'Ciudades']
print("=== RESUMEN POR CATEGORÍA ===")
print(resumen)

# 4. Filtrar productos premium en la categoría líder
categoria_lider = df.groupby('categoria')['total'].sum().idxmax()
df_premium_lider = df[
    (df['categoria'] == categoria_lider) & 
    (df['por_encima_promedio'] == True)
]
print(f"\n=== PRODUCTOS PREMIUM EN CATEGORÍA LÍDER ({categoria_lider}) ===")
print(f"Total de productos: {len(df_premium_lider)}")
print(df_premium_lider[['producto', 'precio', 'precio_promedio_categoria']].head())